In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import h5py

%matplotlib widget
from matplotlib import pyplot as plt
# plt.style.use('fivethirtyeight')

from mpi4py import MPI

import numpy as np

from pyMoBiMP.fenicsx_utils import read_data

from pyMoBiMP.gmsh_utils import dfx_spherical_mesh

from pyMoBiMP.plotting_utils import (
    add_arrow,
    plot_charging_cycle,
    plot_time_sequence,
    PyvistaAnimation,
    animate_time_series)

comm_world = MPI.COMM_WORLD

## Read in the data

In [ ]:
filename = "../simulation_output/CH_4_min_10_particles"

num_particles, t, x_data, u_data, rt_data = read_data(filename)

In [ ]:
[plot_time_sequence((x_data, t, u_data[:, i_part, :, :]), lambda y: np.exp(y) / (1 + np.exp(y))) 
 for i_part in range(num_particles)]

plt.show()

In [ ]:
mu_1_bc = u_data[:, 0, 1, -1]
mu_2_bc = u_data[:, 1, 1, -1]

mu_diff = mu_1_bc - mu_2_bc

plt.figure()

plt.plot(t, mu_diff)

plt.show()

print(np.abs(mu_1_bc).max())
print(np.abs(mu_2_bc).max())
print(np.abs(u_data[:, 0, 1, -1] - u_data[:, 1, 1, -1]).max())

In [ ]:
import dolfinx as dfx
import os

mesh_file = "mesh_3d.xdmf"

try:
    print(f">>> Read mesh from {mesh_file}")
    with dfx.io.XDMFFile(comm_world, mesh_file, "r") as file:
        mesh_3d = file.read_mesh()

except (RuntimeError, FileNotFoundError) as e:
    print(e)

    mesh_3d, _, _ = dfx_spherical_mesh(comm_world, resolution=0.1, optimize=False)

    print(f">>> Write mesh to {mesh_file}")
    with dfx.io.XDMFFile(comm_world, mesh_file, "w") as file:
        file.write_mesh(mesh_3d)

In [ ]:
a = 1.5
b = 0.2
cc = 5.

chem_pot = lambda q: np.log(q / (1 - q)) + a * (1 - 2 * q) + b * np.pi * cc * np.cos(np.pi * cc * q)

rt_data_ = rt_data.copy()

meshes = [mesh_3d, ] * num_particles

anim = PyvistaAnimation(
    (x_data, t, u_data[:, :, :, :]),
    rt_data=rt_data_,
    meshes=meshes,
    c_of_y=lambda y: np.exp(y) / (1 + np.exp(y)),
    f_of_q=chem_pot,
    clim=[0.0, 1.0],
    cmap="fire",
    clipped=True
)

widget = anim.get_slider_widget()

In [ ]:
fig, ax = plt.subplots()

def c_of_y(y):
    return np.exp(y) / (np.exp(y) + 1)

[ax.plot(t, c_of_y(u_data[:, i_part, 0, -1]), label=f"particle_{i_part}") for i_part in range(num_particles)]

ax.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots()

def c_of_y(y):
    return np.exp(y) / (np.exp(y) + 1)

charges = 3 * np.trapz(c_of_y(u_data[:, :, 0, :]) * x_data[:, 0]**2, 
                  x_data[:, 0], axis=-1)

total_charge = charges.sum(axis=1) / num_particles

c_rate = (total_charge[2] - total_charge[0]) / t[2]

plt.plot(t, total_charge, 'k', label='total charge')
[plt.plot(t, charges[:, i_part], label=f"charge_{i_part}") for i_part in range(num_particles)]

ax.legend()

plt.show()

In [ ]:
plt.figure()

plt.plot((total_charge[2:] - total_charge[0]) / t[2:])

plt.show()

In [ ]:
plt.figure()

plt.plot(rt_data[:, 0], rt_data[:, 1])

plt.show()